<a href="https://colab.research.google.com/github/Kritsakorn-64/229352-StatisticalLearning/blob/main/%E0%B8%AA%E0%B8%B3%E0%B9%80%E0%B8%99%E0%B8%B2%E0%B8%82%E0%B8%AD%E0%B8%87_Lab11_Fine_Tuning_Reasoning_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Statistical Learning for Data Science 2 (229352)
#### Instructor: Donlapark Ponnoprat

#### [Course website](https://donlapark.pages.dev/229352/)

## Lab #11

# Fine-Tuning Qwen3 for Thai Text Reasoning

Today, you will learn how to take a pre-trained Large Language Model (LLM) and specialize it for **Thai text reasoning** using the [Thai Reasoning Dataset](https://huggingface.co/datasets/iapp/Thai-R1-Distill-SFT).

We will be using [**Unsloth**](https://docs.unsloth.ai/get-started/all-our-models) to speeds up finetuning and reduces memory usage, making it possible to train in Google Colab.

If GPUs are not available in your Colab, you might want to try these two alternatives:
1. [Kaggle](https://www.kaggle.com)
2. [lightning.ai](https://lightning.ai)

### Installation

In [ ]:
%%capture
import os, re
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    import torch; v = re.match(r"[0-9\.]{3,}", str(torch.__version__)).group(0)
    xformers = "xformers==" + ("0.0.32.post2" if v == "2.8.0" else "0.0.29.post3")
    !pip install --no-deps bitsandbytes accelerate {xformers} peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1,<4.0.0" "huggingface_hub>=0.34.0" hf_transfer
    !pip install --no-deps unsloth
!pip install transformers==4.55.4
!pip install --no-deps trl==0.22.2

### Fine-tuning Qwen3 with Unsloth

#### Transformer in Qwen3 vs GPT-2
<img src="https://substackcdn.com/image/fetch/$s_!GGk9!,f_auto,q_auto:good,fl_progressive:steep/https%3A%2F%2Fsubstack-post-media.s3.amazonaws.com%2Fpublic%2Fimages%2Fa8fa602a-aa4d-4526-9252-c2f09dd5de92_1871x1920.png" alt="transformers" width="600"/>

[Source: [Sebastian Raschka](https://magazine.sebastianraschka.com/p/qwen3-from-scratch)]

In [ ]:
from unsloth import FastLanguageModel
import torch

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Qwen3-14B",
    max_seq_length = 2048,   # Context length - can be longer, but uses more memory
    load_in_4bit = True,     # 4bit uses much less memory
    load_in_8bit = False,    # A bit more accurate, uses 2x memory
    full_finetuning = False, # We have full finetuning now!
)

#### Low-Rank Adaptation (LoRA)
<img src="https://substackcdn.com/image/fetch/$s_!LXL5!,f_auto,q_auto:good,fl_progressive:steep/https%3A%2F%2Fsubstack-post-media.s3.amazonaws.com%2Fpublic%2Fimages%2F7ab94b67-4efa-45e2-9a77-3121d6c88c45_1284x364.png" alt="LoRA" width="600"/>

[Source: [Dasha Herrmannova](https://oneminutenlp.com/p/low-rank-adaptation)]

We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 32,           # Choose any number > 0! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 32,  # Best to choose alpha = rank or rank*2
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,   # We support rank stabilized LoRA
    loftq_config = None,  # And LoftQ
)

<a name="Data"></a>
### Data Prep
Use the [Thai Reasoning Dataset](https://huggingface.co/datasets/iapp/Thai-R1-Distill-SFT) provided by [iApp Technology](https://iapp.co.th/).

In [ ]:
from datasets import load_dataset
reasoning_dataset = load_dataset("iapp/Thai-R1-Distill-SFT", split = "train")

Let's see the structure of both datasets:

In [ ]:
reasoning_dataset

We now convert the reasoning dataset into conversational format:

In [ ]:
def generate_conversation(examples):
    problems  = examples["problem"]
    solutions = examples["reannotated_assistant_content"]
    conversations = []
    for problem, solution in zip(problems, solutions):
        conversations.append([
            {"role" : "user",      "content" : problem},
            {"role" : "assistant", "content" : solution},
        ])
    return { "conversations": conversations, }

In [ ]:
reasoning_conversations = tokenizer.apply_chat_template(
    reasoning_dataset.map(generate_conversation, batched = True)["conversations"],
    tokenize = False,
)

Let's see the first transformed row:

In [ ]:
reasoning_conversations[0]

In [ ]:
print(len(reasoning_conversations))

In [ ]:
import pandas as pd

data = pd.Series(reasoning_conversations)
data.name = "text"

from datasets import Dataset
prepared_dataset = Dataset.from_pandas(pd.DataFrame(data))
prepared_dataset = prepared_dataset.shuffle(seed = 3407)

<a name="Train"></a>
### Train the model
Now let's train our model. We do 60 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`.

In [ ]:
import gc
from trl import SFTTrainer, SFTConfig

# Free some memory
gc.collect()
torch.cuda.empty_cache()

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = prepared_dataset,
    eval_dataset = None, # Can set up evaluation!
    args = SFTConfig(
        dataset_text_field = "text",
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4, # Use GA to mimic batch size!
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 15,
        learning_rate = 2e-4, # Reduce to 2e-5 for long training runs
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        report_to = "none", # Use this for WandB etc
    ),
)

Let's train the model! To resume a training run, set `trainer.train(resume_from_checkpoint = True)`

In [ ]:
trainer_stats = trainer.train()

<a name="Inference"></a>
### Inference
Let's run the model via Unsloth native inference! According to the `Qwen-3` team, the recommended settings for reasoning inference are `temperature = 0.6, top_p = 0.95, top_k = 20`

For normal chat based inference, `temperature = 0.7, top_p = 0.8, top_k = 20`

In [ ]:
FastLanguageModel.for_inference(model)

messages = [
    {"role" : "user", "content" : "จงหาจำนวนเต็ม x และ y ทั้งหมดที่ x^2 - y^2 = 5"}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize = False,
    add_generation_prompt = True, # Must add for generation
    enable_thinking = True, # Enable thinking
)

from transformers import TextStreamer
_ = model.generate(
    **tokenizer(text, return_tensors = "pt").to("cuda"),
    max_new_tokens = 1024, # Increase for longer outputs!
    temperature = 0.7, top_p = 0.8, top_k = 20, # For non thinking
    streamer = TextStreamer(tokenizer, skip_prompt = True),
)

### Exercise 1: Ask the model a challenging math problem. You may increase the value of `max_new_tokens` if the model's answer is too short. Does the model solve your problem correctly?

In [ ]:
### YOUR CODE HERE ###
import re, math
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer
import torch

# พยายามใช้งาน pipeline ที่สร้างไว้แล้วในโน้ตบุ๊ก (ถ้ามี)
generator = None
try:
    generator  # noqa: F401
except NameError:
    generator = None

if generator is None:
    # หากในโน้ตบุ๊กมีตัวแปร model/tokenizer อยู่แล้ว ให้ใช้ต่อทันที
    try:
        model  # noqa: F401
        tokenizer  # noqa: F401
        generator = pipeline(
            "text-generation",
            model=model,
            tokenizer=tokenizer,
            # device=0 if torch.cuda.is_available() else -1 # Removed device argument
        )
    except NameError:
        # กรณีไม่มีโมเดล/โทเคไนเซอร์ที่โหลดไว้:
        # แทนที่ "sshleifer/tiny-gpt2" ด้วย checkpoint ของคุณเองหากออฟไลน์
        fallback_model = "sshleifer/tiny-gpt2"
        tokenizer = AutoTokenizer.from_pretrained(fallback_model)
        model = AutoModelForCausalLM.from_pretrained(fallback_model)
        generator = pipeline(
            "text-generation",
            model=model,
            tokenizer=tokenizer,
            # device=0 if torch.cuda.is_available() else -1 # Removed device argument
        )

def ask_model(prompt, max_new_tokens=160, temperature=0.2):
    out = generator(
        prompt,
        max_new_tokens=max_new_tokens,
        do_sample=False,
        temperature=temperature,
        eos_token_id=tokenizer.eos_token_id
    )
    return out[0]["generated_text"]

# โจทย์คณิตศาสตร์: หาค่า LCM ของ 420 และ 378
# กำชับให้โมเดลพิมพ์ผลลัพธ์เชิงตัวเลขต่อท้าย 'Answer:' เพื่อให้ง่ายต่อการตรวจ
problem_text = (
    "You are a careful math tutor. Solve step by step, then give only the final numeric result "
    "after the word 'Answer:'.\n\n"
    "Problem: What is the least common multiple (LCM) of 420 and 378?\n"
    "Answer:"
)

# คำตอบจริง (ground truth) ด้วย Python
gt = (420 * 378) // math.gcd(420, 378)

# เรียกโมเดล (ปรับ max_new_tokens ได้)
generated = ask_model(problem_text, max_new_tokens=160, temperature=0.2)

print("=== Model Output ===")
print(generated)

# ดึงตัวเลขหลัง 'Answer:'
matches = re.findall(r"Answer\\s*:\\s*([-+]?\\d+)", generated)
pred = int(matches[-1]) if matches else None

print("\n=== Automatic Check ===")
print("Ground truth:", gt)
print("Predicted   :", pred)
if pred is None:
    print("Result: Could not parse an explicit numeric answer after 'Answer:'. "
          "Consider increasing max_new_tokens or refining the instruction.")
else:
    print("Result:", "CORRECT ✅" if pred == gt else "INCORRECT ❌")

**Answer to Exercise 1**

-

### Visualizing Attention

### Exercise 2: Fill in the ##TODO## part in the `visualize_attention()` function below to calculate the product between the query matrix `Q` and the key matrix `K`.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Define the helper function to perform the rotation
def rotate_half(x):
    """Rotates half the hidden dimensions of the input."""
    x1 = x[..., : x.shape[-1] // 2]
    x2 = x[..., x.shape[-1] // 2 :]
    return torch.cat((-x2, x1), dim=-1)

# Define a global dictionary to store the attention matrix
attention_viz_data = {}

def visualize_attention(model, tokenizer, prompt, layer_number):
    """
    Visualizes the QK^T attention matrix for a given prompt and layer,

    Args:
        model: The Unsloth Qwen3 14B model.
        tokenizer: The tokenizer for the model.
        prompt (str): The input text prompt.
        layer_number (int): The decoder layer number to visualize.
    """
    attention_viz_data.clear()

    # 1. Get a reference to the specific attention layer
    try:
        attention_layer = model.base_model.model.model.layers[layer_number].self_attn
    except AttributeError as e:
        print(f"Could not find the specified layer. Please double-check the model architecture path. Error: {e}")
        return

    # 2. Store the original forward method
    original_forward = attention_layer.forward

    def new_forward(self, hidden_states, *args, **kwargs):
        """A new forward method to capture the QK^T matrix."""

        bsz, q_len, _ = hidden_states.size()

        # Project to Q, K
        Q_all_heads = self.q_proj(hidden_states)
        K_all_heads = self.k_proj(hidden_states)

        # Reshape for multi-head attention. Note the different head counts.
        Q_all_heads = Q_all_heads.view(bsz, q_len, self.config.num_attention_heads, self.head_dim).transpose(1, 2)
        K_all_heads = K_all_heads.view(bsz, q_len, self.config.num_key_value_heads, self.head_dim).transpose(1, 2)

        # Get rotary embeddings
        cos, sin = self.rotary_emb(K_all_heads, seq_len=q_len)

        # Apply rotary embeddings directly
        Q_all_heads = (Q_all_heads * cos) + (rotate_half(Q_all_heads) * sin)
        K_all_heads = (K_all_heads * cos) + (rotate_half(K_all_heads) * sin)

        # Repeat the key heads to match the number of query heads.
        num_key_value_groups = self.config.num_attention_heads // self.config.num_key_value_heads
        if num_key_value_groups > 1:
            K_all_heads = K_all_heads.repeat_interleave(num_key_value_groups, dim=1)

        # Now, the dimensions match for matrix multiplication:
        # Q_all_heads: [1, 40, 29, 128] = [batch_size, num_heads, seq_len, head_dim]
        # K_all_heads: [1, 40, 29, 128] = [batch_size, num_heads, seq_len, head_dim]

        Q = Q_all_heads[0, 0]  # Queries of the first head
        K = K_all_heads[0, 0]  # Keys of the first head

        ########## TODO: CALCULATE QK^T ############
        attention = torch.matmul(Q, K.transpose(-1, -2))

        # Store the matrix for visualization
        attention_viz_data['qk_t'] = attention.detach()

        # Call the original forward method to ensure the model's computation continues normally
        return original_forward(hidden_states, *args, **kwargs)

    # 3. Monkey-patch the layer's forward method
    try:
        attention_layer.forward = new_forward.__get__(attention_layer, type(attention_layer))

        # 4. Prepare and run the model
        inputs = tokenizer(prompt, return_tensors="pt")
        input_ids = inputs.input_ids.to(model.device)

        with torch.no_grad():
            _ = model(input_ids)

    finally:
        # 5. Restore the original forward method
        attention_layer.forward = original_forward

    # 6. Visualize the captured data
    if 'qk_t' in attention_viz_data:
        # We visualize the scores for the first head of the first batch item
        qk_matrix = attention_viz_data['qk_t'].cpu().float().numpy()

        tokens = tokenizer.convert_ids_to_tokens(input_ids[0])
        tokens = [token[1:] if token[0] == 'Ġ' else token for token in tokens]

        plt.figure(figsize=(12, 10))
        im = plt.imshow(qk_matrix, cmap='viridis', interpolation='nearest')
        plt.colorbar(im, label='Attention Score (Logits)')
        plt.xticks(ticks=np.arange(len(tokens)), labels=tokens, rotation=90)
        plt.yticks(ticks=np.arange(len(tokens)), labels=tokens)
        plt.xlabel("Key Tokens")
        plt.ylabel("Query Tokens")
        plt.title(f"QK^T Matrix Visualization for Layer {layer_number}, Head 0")
        plt.tight_layout()
        plt.show()
    else:
        print(f"Failed to capture the QK^T matrix for layer {layer_number}.")

In [ ]:
example_text = "Find all integers x and y such that x^2 - y^2 = 5."

visualize_attention(model, tokenizer, example_text, 0)

### Exercise 3: Choose your own example text. Find **two** pairs of two distinct words with the highest attention values. Why do you think those two pairs have the highest attention values?

In [ ]:
### YOUR CODE HERE ###
import math
import torch
import numpy as np
from typing import List, Tuple, Dict
from transformers import (
    AutoTokenizer,
    AutoModel,
    AutoModelForCausalLM,
)
torch.set_grad_enabled(False)
attention_weights_data = {}

def capture_attention_hook(module, input, output):
    if isinstance(output, tuple) and len(output) > 0:
        attention_weights_data['attention'] = output[0].detach().cpu()

# 1) เลือกโมเดล/โทเคไนเซอร์
_model, _tokenizer = None, None
try:
    model  # noqa: F401
    tokenizer  # noqa: F401
    _model, _tokenizer = model, tokenizer
except NameError:
    _model, _tokenizer = None, None

if _model is None or _tokenizer is None:
    # Fallback to a different model if the Unsloth model is not available
    fallback_name = "distilbert-base-uncased"  # encoder; attention ใช้ง่าย
    _tokenizer = AutoTokenizer.from_pretrained(fallback_name, use_fast=True)
    _model = AutoModel.from_pretrained(fallback_name)

hooks = []
if hasattr(_model, 'base_model') and hasattr(_model.base_model, 'model') and hasattr(_model.base_model.model, 'model') and hasattr(_model.base_model.model.model, 'layers'):
    for layer in _model.base_model.model.model.layers:
        if hasattr(layer, 'self_attn'):
            hook = layer.self_attn.register_forward_hook(capture_attention_hook)
            hooks.append(hook)
elif hasattr(_model, 'encoder') and hasattr(_model.encoder, 'layer'): # For models like DistilBERT
     for layer in _model.encoder.layer:
        if hasattr(layer, 'attention') and hasattr(layer.attention, 'q_lin'): # Check for a common pattern in attention layers
            hook = layer.attention.register_forward_hook(capture_attention_hook)
            hooks.append(hook)
else:
    print("Could not automatically find attention layers. Please inspect the model structure.")

# 2) เลือกข้อความตัวอย่าง (ปรับได้)
text = (
    "Because the autoclave cycle failed yesterday, the team repeated the sterilization today, "
    "verifying lethality with biological indicators at three worst-case positions."
)

# 3) เข้ารหัสและขอ attention
enc = _tokenizer(
    text,
    return_tensors="pt",
    return_offsets_mapping=True,
    return_special_tokens_mask=True
)

# Call the model's forward pass to trigger the hooks
try:
    with torch.no_grad():
        _model(**{k: v for k, v in enc.items() if k in ("input_ids","attention_mask")})
except Exception as e:
    print(f"Error during model forward pass: {e}")


# Remove the hooks after the forward pass
for hook in hooks:
    hook.remove()

# Get the captured attention weights
attentions = attention_weights_data.get('attention', None)

if attentions is None:
    print("Failed to capture attention weights. Ensure the hook registration logic is correct for the model architecture.")
    # Fallback to a dummy attention matrix if nothing was captured
    seq_len = enc["input_ids"].shape[-1]
    att_mean = torch.ones((seq_len, seq_len)) / seq_len # Just a placeholder
else:
    # Average attention weights across layers and heads
    # shape → (seq, seq)
    # attentions shape is (batch_size, num_heads, seq_len, seq_len)
    att_mean = attentions.mean(dim=(0, 1))[0] # Average over batch and heads

# 4) รวม token → word
word_ids = enc.word_ids()  # list len = seq
tokens = _tokenizer.convert_ids_to_tokens(enc["input_ids"][0].tolist())

# สร้างกลุ่ม index ของ token ต่อ word
word_to_tokidxs: Dict[int, List[int]] = {}
for tidx, wid in enumerate(word_ids):
    if wid is None:
        continue
    word_to_tokidxs.setdefault(wid, []).append(tidx)

# สร้างคำ (string) ต่อ word-id ด้วย offset mapping
offsets = enc["offset_mapping"][0].tolist()  # [(start,end), ...]
words: List[str] = []
# Ensure words are indexed from 0 to W-1
sorted_word_ids = sorted(set([w for w in word_ids if w is not None]))
word_id_to_idx = {wid: idx for idx, wid in enumerate(sorted_word_ids)}

for wid in sorted_word_ids:
    # ใช้ช่วง offset ของ token แรก→สุดท้ายของคำ
    tok_ids = word_to_tokidxs[wid]
    start = min(offsets[i][0] for i in tok_ids)
    end   = max(offsets[i][1] for i in tok_ids)
    words.append(text[start:end])

# 5) คำนวณ attention ระหว่าง "คำ"
def word_pair_attention(i: int, j: int) -> float:
    if i == j:
        return float("nan")

    # Get the token indices for the word indices
    toks_i = word_to_tokidxs[sorted_word_ids[i]]
    toks_j = word_to_tokidxs[sorted_word_ids[j]]

    # Ensure att_mean is a numpy array for easier indexing
    if isinstance(att_mean, torch.Tensor):
        submat = att_mean[np.ix_(toks_i, toks_j)]
        return float(submat.mean().item())
    elif isinstance(att_mean, np.ndarray):
        submat = att_mean[np.ix_(toks_i, toks_j)]
        return float(submat.mean())
    else:
        return float("nan")


W = len(words)
pairs: List[Tuple[int,int,float]] = []
for i in range(W):
    for j in range(W):
        if i == j:
            continue
        score = word_pair_attention(i, j)
        if not math.isnan(score):
            pairs.append((i, j, score))

# เรียงลำดับจาก attention สูงสุด
pairs_sorted = sorted(pairs, key=lambda x: x[2], reverse=True)
top2 = pairs_sorted[:2]

# 6) รายงานผล
def pretty_word(w: str) -> str:
    # ทำความสะอาดเว้นวรรค/จุดคั่น
    return w.strip()

print("=== Example text ===")
print(text, "\n")

print("=== Top-2 word→word attention pairs (directional) ===")
for rank, (i, j, s) in enumerate(top2, start=1):
    wi, wj = pretty_word(words[i]), pretty_word(words[j])
    print(f"{rank}) ({i}:'{wi}') → ({j}:'{wj}')  score={s:.6f}")

**Answers to Exercise 3**

-

<a name="Save"></a>
### Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!

In [ ]:
model.save_pretrained("lora_model")  # Local saving
tokenizer.save_pretrained("lora_model")
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving

Now if you want to load the LoRA adapters we just saved for inference, set `False` to `True`:

In [ ]:
if False:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = 2048,
        load_in_4bit = True,
    )

### Saving to float16 for VLLM

We also support saving to `float16` directly. Select `merged_16bit` for float16 or `merged_4bit` for int4. We also allow `lora` adapters as a fallback. Use `push_to_hub_merged` to upload to your Hugging Face account! You can go to https://huggingface.co/settings/tokens for your personal tokens.

In [ ]:
# Merge to 16bit
if False:
    model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)
if False: # Pushing to HF Hub
    model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_16bit", token = "")

# Merge to 4bit
if False:
    model.save_pretrained_merged("model", tokenizer, save_method = "merged_4bit",)
if False: # Pushing to HF Hub
    model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_4bit", token = "")

# Just LoRA adapters
if False:
    model.save_pretrained("model")
    tokenizer.save_pretrained("model")
if False: # Pushing to HF Hub
    model.push_to_hub("hf/model", token = "")
    tokenizer.push_to_hub("hf/model", token = "")
